In [1]:
import numpy as np
import cupy as cp
import time

In [2]:
vector_sum_kernel = cp.RawKernel(r'''
extern "C" __global__
void vector_sum(const float* A, float* result, int N) {
    // Calculate global thread index
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Shared memory for reduction
    __shared__ float cache[256];

    float temp = 0.0;
    if (idx < N) {
        temp = A[idx];
    } else {
        temp = 0.0;
    }
    
    // Store value in shared memory
    cache[threadIdx.x] = temp;
    __syncthreads();

    // Perform reduction in shared memory
    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (threadIdx.x < stride) {
            cache[threadIdx.x] += cache[threadIdx.x + stride];
        }
        __syncthreads();
    }

    // Write the block result to global memory
    if (threadIdx.x == 0) {
        atomicAdd(result, cache[0]);
    }
}
''', 'vector_sum')


In [3]:
def vector_sum_cpu(vector):
    result_cpu = 0
    for element in vector:
        result_cpu += element
    return result_cpu

In [4]:
def vector_sum_gpu(vector):
    N = vector.size
    A_gpu = cp.asarray(vector, dtype=cp.float32)

    # Prepare the result array (single float for the final total)
    result_gpu = cp.zeros(1, dtype=cp.float32)
    
    # Define block and grid sizes
    block_size = 256
    grid_size = (N + block_size - 1) // block_size

    # Launch the kernel
    vector_sum_kernel((grid_size,), (block_size,), (A_gpu, result_gpu, N))
    
    # Retrieve the result
    return cp.asnumpy(result_gpu[0])  # Convert back to CPU


In [5]:
vector_size = 1000000  # Change this value as needed
vector = np.random.rand(vector_size).astype(np.float32)

# Sum on CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu

print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time:.6f} секунд")

# Sum on GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu

print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time:.6f} секунд")

Сумма на CPU: 499814.68238035165, Время выполнения на CPU: 0.093999 секунд
Сумма на GPU: 499813.6875, Время выполнения на GPU: 0.558191 секунд


In [19]:
vector_size = 1000000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 500127.6603554417, Время выполнения на CPU: 0.05231952667236328 секунд
Сумма на GPU: 500128.4375, Время выполнения на GPU: 0.0020008087158203125 секунд


In [17]:
vector_size = 100000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 49995.38013009982, Время выполнения на CPU: 0.005003452301025391 секунд
Сумма на GPU: 49995.38013009983, Время выполнения на GPU: 0.00499415397644043 секунд


In [18]:
vector_size = 1000000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 500320.66229143896, Время выполнения на CPU: 0.05040907859802246 секунд
Сумма на GPU: 500320.66229143133, Время выполнения на GPU: 0.00500035285949707 секунд


In [21]:
vector_size = 10000000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 4999503.079631773, Время выполнения на CPU: 0.500321626663208 секунд
Сумма на GPU: 4999503.079632387, Время выполнения на GPU: 0.09285140037536621 секунд


In [20]:
vector_size = 100000000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 50001617.07631788, Время выполнения на CPU: 5.042378187179565 секунд
Сумма на GPU: 50001617.07631542, Время выполнения на GPU: 0.6173648834228516 секунд


In [22]:
vector_size = 100000000
vector = np.random.rand(vector_size)

# Сложение на CPU
start_time_cpu = time.time()
result_cpu = vector_sum_cpu(vector)
end_time_cpu = time.time()

cpu_time = end_time_cpu - start_time_cpu
print(f"Сумма на CPU: {result_cpu}, Время выполнения на CPU: {cpu_time} секунд")

# Сложение на GPU
start_time_gpu = time.time()
result_gpu = vector_sum_gpu(vector)
end_time_gpu = time.time()

gpu_time = end_time_gpu - start_time_gpu
print(f"Сумма на GPU: {result_gpu}, Время выполнения на GPU: {gpu_time} секунд")    

Сумма на CPU: 49998562.984551504, Время выполнения на CPU: 4.891952991485596 секунд
Сумма на GPU: 49998562.98455838, Время выполнения на GPU: 0.4043457508087158 секунд
